In [114]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from tqdm.auto import tqdm
import glob
import os
import sys

# pip3 install --user PyMuPDF
import fitz

In [115]:
def parse_stream(stream):
    data_raw = []
    data_transformed = []
    rotparams = None
    npatches = 0
    for line in stream.splitlines():
        if line.endswith(" cm"):
            # page 146 of https://www.adobe.com/content/dam/acom/en/devnet/pdf/pdfs/pdf_reference_archives/PDFReference.pdf
            rotparams = list(map(float,line.split()[:-1]))
        elif line.endswith(" l"):
            x,y = list(map(float,line.split()[:2]))
            a,b,c,d,e,f = rotparams
            xp = a*x+c*y+e
            yp = b*x+d*y+f
            data_transformed.append([xp,yp])
            data_raw.append([x,y])
        elif line.endswith(" m"):
            npatches += 1
        else:
            pass
    data_raw = np.array(data_raw)
    if len(data_raw) < 1:
        return dict(data=np.array(data_raw), npatches=npatches, good=False)
    base_x, base_y = data_raw[-1]
    good = False
    if base_x == 0.:
        data_raw[:,1] = base_y - data_raw[:,1]
        data_raw[:,1] *= 100/60.
        data_raw = data_raw[data_raw[:,1]!=0.]
        if npatches == 1: good = True
    return dict(data=np.array(data_raw), npatches=npatches, good=good)

In [116]:
def parse_page(doc, page_index, verbose=False):
    categories = [
        "Retail & recreation",
        "Grocery & pharmacy",
        "Parks",
        "Transit stations",
        "Workplace",
        "Residential",
    ]

    counties = []
    curr_county = None
    curr_category = None
    data = defaultdict(lambda: defaultdict(list))
    pagetext = doc.getPageText(page_index)
    lines = pagetext.splitlines()
    tickdates = list(filter(lambda x:len(x.split())==3, set(lines[-10:])))
    for line in lines:
        # if we encountered a category, add to dict, otherwise
        # push all seen lines into the existing dict entry
        if any(line.startswith(c) for c in categories):
            curr_category = line
        elif curr_category:
            data[curr_county][curr_category].append(line)

        # If it doesn't match anything, then it's a county name
        if (all(c not in line for c in categories)
            and ("compared to baseline" not in line)
            and ("Not enough data" not in line)
           ):
            # saw both counties already
            if len(data.keys()) == 2: break
            counties.append(line)
            curr_county = line

    newdata = {}
    for county in data:
        newdata[county] = {}
        for category in data[county]:
            # if the category text ends with a space, then there was a star/asterisk there
            # indicating lack of data. we skip these.
            if category.endswith(" "): continue
            temp = [x for x in data[county][category] if "compared to baseline" in x]
            if not temp: continue
            percent = int(temp[0].split()[0].replace("%",""))
            newdata[county][category.strip()] = percent
    data = newdata

    tomatch = []
    for county in counties:
        for category in categories:
            if category in data[county]:
                tomatch.append([county,category,data[county][category]])
    if verbose:
        print(len(tomatch))
        print(data)
    
    goodplots = []
    xrefs = sorted(doc.getPageXObjectList(page_index), key=lambda x:int(x[1].replace("X","")))
    for i, xref in enumerate(xrefs):
        stream = doc.xrefStream(xref[0]).decode()
        info = parse_stream(stream)
        if not info["good"]: continue
        goodplots.append(info)
    if verbose:
        print(len(goodplots))
    
    ret = []
    
    if len(tomatch) != len(goodplots):
        return ret
    
    
    for m,g in zip(tomatch,goodplots):
        xs = g["data"][:,0]
        ys = g["data"][:,1]
        maxys = ys[np.where(xs==xs.max())[0]]
        maxy = maxys[np.argmax(np.abs(maxys))]
        
        
        # parsed the tick date labels as text. find the min/max (first/last)
        # and make evenly spaced dates, one per day, to assign to x values between
        # 0 and 200 (the width of the plots).
        ts = list(map(lambda x: pd.Timestamp(x.split(None,1)[-1] + ", 2020"), tickdates))
        low, high = min(ts), max(ts)
        dr = list(map(lambda x:str(x).split()[0], pd.date_range(low, high, freq="D")))
        lutpairs = list(zip(np.linspace(0,200,len(dr)),dr))

        dates = []
        values = []
        asort = xs.argsort()
        xs = xs[asort]
        ys = ys[asort]
        for x, y in zip(xs,ys):
            date = min(lutpairs, key=lambda v:abs(v[0]-x))[1]
            dates.append(date)
            values.append(round(y,3))

        ret.append(dict(
            county=m[0],category=m[1],change=m[2],
            values=values,
            dates=dates,
            changecalc=maxy,
        ))
    return ret


In [117]:
def parse_page_country(country, doc, page_index, verbose=False):
    categories = [
        "Retail & recreation",
        "Grocery & pharmacy",
        "Parks",
        "Transit stations",
        "Workplace",
        "Residential",
    ]

    current_category = None
    data = defaultdict(lambda: defaultdict(list))
    
    for category in categories:
        data[category] = []
        
    pagetext = doc.getPageText(page_index)
    lines = pagetext.splitlines()
    tickdates = list(filter(lambda x:len(x.split())==3, set(lines[-10:])))
    for line in range(len(lines)):
        # if we encountered a category, add to dict, otherwise
        # push all seen lines into the existing dict entry
        for c in categories:
            if lines[line].startswith(c):
                current_category = c
        if current_category and "Not enough data" not in lines[line-1] and "Not enough data" not in lines[line-1]:
            data[current_category].append(lines[line-1] + " " + lines[line])

    newdata = {}
    print(data)
    for category in data:
        # if the category text ends with a space, then there was a star/asterisk there
        # indicating lack of data. we skip these.
        if category.endswith(" "):
            continue
        temp = [x for x in data[category] if "compared to baseline" in x]
        if not temp: continue
        try:
            percent = int(temp[0].split()[0].replace("%",""))
        except:
            print(temp[0])
        newdata[category.strip()] = percent
    data = newdata
    print(data)

    tomatch = []
    for category in categories:
        if category in data:
            tomatch.append([country,category,data[category]])

    if verbose:
        print(len(tomatch))
        print(data)
    
    goodplots = []
    xrefs = sorted(doc.getPageXObjectList(page_index), key=lambda x:int(x[1].replace("X","")))
    for i, xref in enumerate(xrefs):
        stream = doc.xrefStream(xref[0]).decode()
        info = parse_stream(stream)
        if not info["good"]: continue
        goodplots.append(info)
    if verbose:
        print(len(goodplots))
    
    ret = []
    
    if len(tomatch) != len(goodplots):
        print("failed")
        return ret
    
    
    for m,g in zip(tomatch,goodplots):
        xs = g["data"][:,0]
        ys = g["data"][:,1]
        maxys = ys[np.where(xs==xs.max())[0]]
        maxy = maxys[np.argmax(np.abs(maxys))]
        
        
        # parsed the tick date labels as text. find the min/max (first/last)
        # and make evenly spaced dates, one per day, to assign to x values between
        # 0 and 200 (the width of the plots).
        ts = []
        for x in tickdates:
            print(x)
            try:
                ts.append(pd.Timestamp(x.split(None, 1)[-1] + ", 2020"))
            except:
                print(x)
        # print(list(map(lambda x: x.split(None,1)[-1] + ", 2020", tickdates)))
        # ts = list(map(lambda x: pd.Timestamp(x.split(None,1)[-1] + ", 2020"), tickdates))
        low, high = min(ts), max(ts)
        dr = list(map(lambda x:str(x).split()[0], pd.date_range(low, high, freq="D")))
        lutpairs = list(zip(np.linspace(0,200,len(dr)),dr))

        dates = []
        values = []
        asort = xs.argsort()
        xs = xs[asort]
        ys = ys[asort]
        for x, y in zip(xs,ys):
            date = min(lutpairs, key=lambda v:abs(v[0]-x))[1]
            dates.append(date)
            values.append(round(y,3))

        ret.append(dict(
            county=m[0],category=m[1],change=m[2],
            values=values,
            dates=dates,
            changecalc=maxy,
        ))
    return ret

In [118]:
def parse_state(state):
    document = fitz.Document(f"mobilityData/US/2020-04-05_US_{state}_Mobility_Report_en.pdf")
    data = []
    for i in range(2,document.pageCount-1):
        for entry in parse_page(document, i):
            entry["state"] = state
            entry["page"] = i
            data.append(entry)
    df = pd.DataFrame(data)
    return df[["state","county","category","change","changecalc","dates", "values","page"]]

In [119]:
def parse_country(country):
    print(country)
    document = fitz.Document(f"mobilityData/2020-04-05_{country}_Mobility_Report_en.pdf")
    data = []
    for i in range(0, 2):
        for entry in parse_page_country(country, document, i):
            entry["state"] = country
            entry["page"] = i
            data.append(entry)
    df = pd.DataFrame(data)
    return df[["state", "category", "change", "changecalc", "dates", "values", "page"]]

In [120]:
import glob
states = [x.split("_US_",1)[1].split("_Mobility",1)[0] for x in glob.glob("mobilityData/US/*.pdf")]
countries = [x[24:26] for x in glob.glob("mobilityData/*.pdf")]

# for state in tqdm(states):
#
#     df = parse_state(state)
#     data = []
#
#     for i,row in df.iterrows():
#         # do a little clean up and unstack the dates/values as separate rows
#         dorig = dict()
#         dorig["state"] = row["state"].replace("_"," ")
#         dorig["county"] = row["county"]
#         dorig["category"] = row["category"].replace(" & ","/").replace(" ","").lower()
#         dorig["page"] = row["page"]
#         dorig["change"] = row["change"]
#         dorig["changecalc"] = row["changecalc"]
#         for x,y in zip(row["dates"],row["values"]):
#             d = dorig.copy()
#             d["date"] = x
#             d["value"] = y
#             data.append(d)
#     df = pd.DataFrame(data)
#     df.to_json("data/" + state + ".json", orient="records", indent=2)


for country in tqdm(countries):
    dataFrame = parse_country(country)
    data = []

    for i, row in dataFrame.iterrows():
        dorig = dict()
        dorig["state"] = row["state"].replace("_", " ")
        dorig["category"] = row["category"].replace(" & ","/").replace(" ","").lower()
        dorig["page"] = row["page"]
        dorig["change"] = row["change"]
        dorig["changecalc"] = row["changecalc"]
        for date, value in zip(row["dates"], row["values"]):
            d = dorig.copy()
            d["date"] = date
            d["value"] = value
            data.append(d)
    dataFrame = pd.DataFrame(data)
    dataFrame.to_json("data/" + country + ".json", orient="records", indent=2)


AE
defaultdict(<function parse_page_country.<locals>.<lambda> at 0x00000155C44DA158>, {'Retail & recreation': ['we calculate these trends and preserve privacy, read About this data. Retail & recreation', 'Retail & recreation -61%', '-61% compared to baseline', 'compared to baseline Sun Feb 23', 'Sun Feb 23 Sun Mar 15', 'Sun Mar 15 Sun Apr 5', 'Sun Apr 5 -80%', '-80% -40%', '-40% Baseline', 'Baseline +40%', '+40% +80%', '+80% Mobility trends for places like restaurants,', 'Mobility trends for places like restaurants, cafes, shopping centers, theme parks,', 'cafes, shopping centers, theme parks, museums, libraries, and movie theaters.'], 'Grocery & pharmacy': ['museums, libraries, and movie theaters. Grocery & pharmacy', 'Grocery & pharmacy -30%', '-30% compared to baseline', 'compared to baseline Sun Feb 23', 'Sun Feb 23 Sun Mar 15', 'Sun Mar 15 Sun Apr 5', 'Sun Apr 5 -80%', '-80% -40%', '-40% Baseline', 'Baseline +40%', '+40% +80%', '+80% Mobility trends for places like grocery', 'Mobi

UnboundLocalError: local variable 'percent' referenced before assignment